# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Publish-Subscribe (Pub-Sub)

**Subtopic** - Designing a real-time notification system using the Pub-Sub pattern

**Use Case** - Implement a schema evolution handler that dynamically translates incoming JSON payloads to the latest version using a migration map before publishing to the event bus.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

## Problem Overview

You are required to implement a **rule-based schema evolution handler** that migrates a JSON payload from its current version to a specified target version. The migration is driven by a set of **declarative rules** (not functions) that describe how to transform the data between schema versions.

The migration supports **both forward (upgrade)** and **backward (downgrade)** transformations. Each migration step is defined by one or more rules. In addition to simple field operations (`rename`, `add`, `remove`, `set`), your solution must also support **list transformations** - applying a set of rules to each element of a list in the payload.

In this extended version, your handler must also support **version history storage** for each payload. As each version is created during migration, it must be **snapshotted and stored**. Later, the user should be able to **roll back** the payload to any previously stored version **without applying downgrade rules** - the system should restore it from the stored version state.



## Input Format

Your system must support the following operations:

### Operation 1: `migrate_payload`

Arguments:

* `payload: dict`

  * Must contain:

    * `"version"`: a string like `"v2"`, indicating the current schema version.
    * `"data"`: a dictionary representing the structured payload.
* `migration_rules: dict`

  * Keys are strings representing migration steps (e.g., `"v2_to_v3"`).
  * Values are lists of rules to apply in order.
* `target_version: str`

  * Desired schema version to migrate to.

### Operation 2: `rollback_payload`

Arguments:

* `payload_id: str` – A unique identifier for the payload.
* `target_version: str` – The version to roll back to.



## Output Format

* For `migrate_payload`:
  A dictionary with:

  * `"version"`: the new version.
  * `"data"`: the transformed data after all rules have been applied.

* For `rollback_payload`:
  A dictionary representing the full snapshot of the payload in the specified version.



## Supported Rule Types

### Field-Level Rules:

* `{ "op": "rename", "from": str, "to": str }`
* `{ "op": "add", "field": str, "value": Any }`
* `{ "op": "remove", "field": str }`
* `{ "op": "set", "field": str, "value": Any }`

### List Transformation Rule:

* `{ "op": "map", "path": str, "rules": list }`

  * `path`: dot-separated path to a list inside the payload.
  * `rules`: list of the above rule types applied to each element in the list.



## Examples

### Example 1: Upgrade with snapshot storage

```python
# Initial state
payload = {
  "version": "v1",
  "data": {
    "user_id": 101,
    "items": [
      { "product": "A", "price_cents": 1200 },
      { "product": "B", "price_cents": 800 }
    ]
  }
}

migration_rules = {
  "v1_to_v2": [
    { "op": "map", "path": "items", "rules": [
      { "op": "rename", "from": "price_cents", "to": "price" },
      { "op": "add", "field": "currency", "value": "USD" }
    ]}
  ],
  "v2_to_v3": [
    { "op": "rename", "from": "user_id", "to": "uid" }
  ]
}

migrate_payload(payload, migration_rules, "v3", payload_id="order#1")
```

**Internal Storage after migration:**

```python
{
  "order#1": {
    "v1": {... original data ...},
    "v2": {... after first rule set ...},
    "v3": {
      "version": "v3",
      "data": {
        "uid": 101,
        "items": [
          { "product": "A", "price": 1200, "currency": "USD" },
          { "product": "B", "price": 800, "currency": "USD" }
        ]
      }
    }
  }
}
```

### Example 2: Rollback

```python
rollback_payload("order#1", "v1")
```

**Output:**

```python
{
  "version": "v1",
  "data": {
    "user_id": 101,
    "items": [
      { "product": "A", "price_cents": 1200 },
      { "product": "B", "price_cents": 800 }
    ]
  }
}
```

# Requirements


## Explicit Requirements

* Migration steps must be applied sequentially from current version to target version.
* Snapshots of each intermediate and final version must be stored under a unique payload ID.
* Rollback must return a deep copy of the stored version, not a re-transformation.
* `rollback_payload` must raise an error if the version is not stored.
* Input payloads should not be mutated in-place.



## Implicit Requirements

* Versions follow `"vN"` where `N` is a positive integer.
* Migration steps are only defined between adjacent versions.
* Path resolution for list operations supports dot notation.
* List items are treated as independent dictionaries during transformation.
* Stored versions must remain immutable after insertion.



## Edge Cases

* Rollback to the current version - should return a copy of the stored version.
* Rollback to a version that was never migrated to - should raise an error.
* Migrate to same version - should return unchanged payload, but still stored.
* Migration step missing - must raise error.
* List path points to empty or missing field - should be a no-op.
* Rollback to a version with invalidated fields (e.g., missing keys) - return as-is from stored snapshot.



## Function Signatures

```python
def migrate_payload(
    payload: dict,
    migration_rules: dict,
    target_version: str,
    payload_id: str
) -> dict:
    """
    Migrates the input payload to the target version using rule-based transformations.
    Stores intermediate states keyed by payload_id and version.
    """

def rollback_payload(
    payload_id: str,
    target_version: str
) -> dict:
    """
    Returns a previously stored version of a payload by ID and version.
    Does not perform any transformation.
    """
```



## Constraints

* Versions are labeled "vN" where N in [1, 20].
* Migration rules exist only between adjacent versions.
* A payload's version history will never exceed 20 stored versions.
* Payloads are stored per ID and version as deep copies.
* All stored data fits in memory (<= 1 MB per payload).
* All operations complete in under 1 second.
* Rules are well-formed; data may be incomplete or contain missing fields.
* Only lists of dictionaries are allowed as targets of `map` rules.

In [ ]:
# code

import copy
import re

# In-memory storage for payload version snapshots
_snapshot_store = {}


def migrate_payload(payload: dict, migration_rules: dict, target_version: str, payload_id: str) -> dict:
    """
    Migrates the input payload to the target version using rule-based transformations.
    Stores intermediate states keyed by payload_id and version.

    Args:
        payload (dict): The input payload containing 'version' and 'data'.
        migration_rules (dict): Migration steps with rules for each adjacent version.
        target_version (str): The schema version to migrate to.
        payload_id (str): Unique identifier for storing snapshots.

    Returns:
        dict: The transformed payload at the target version.

    Raises:
        ValueError: If required migration steps are missing.
    """
    if not isinstance(payload, dict):
        raise ValueError("Payload must be a dictionary.")

    version_pattern = r"^v(\d+)$"
    current_match = re.match(version_pattern, payload.get("version", ""))
    target_match = re.match(version_pattern, target_version)

    if not current_match or not target_match:
        raise ValueError("Invalid version format. Must be like 'v1', 'v2', ...")

    current_version = int(current_match.group(1))
    target_version_num = int(target_match.group(1))

    if payload_id not in _snapshot_store:
        _snapshot_store[payload_id] = {}

    # Deep copy the original
    current_payload = copy.deepcopy(payload)
    _snapshot_store[payload_id][f"v{current_version}"] = copy.deepcopy(current_payload)

    step = 1 if target_version_num > current_version else -1

    for v in range(current_version, target_version_num, step):
        from_version = f"v{v}"
        to_version = f"v{v + step}"
        direction = f"{from_version}_to_{to_version}"

        if direction not in migration_rules:
            raise ValueError(f"Missing migration rule for: {direction}")

        rules = migration_rules[direction]
        current_payload = apply_rules(current_payload, rules)
        current_payload["version"] = to_version
        _snapshot_store[payload_id][to_version] = copy.deepcopy(current_payload)

    return current_payload


def rollback_payload(payload_id: str, target_version: str) -> dict:
    """
    Returns a previously stored version of a payload by ID and version.
    Does not perform any transformation.

    Args:
        payload_id (str): Unique identifier for the payload.
        target_version (str): The schema version to restore.

    Returns:
        dict: The stored payload snapshot.

    Raises:
        KeyError: If the version is not available for the given ID.
    """
    if payload_id not in _snapshot_store:
        raise KeyError(f"No payload found for ID: {payload_id}")

    if target_version not in _snapshot_store[payload_id]:
        raise KeyError(f"No version '{target_version}' stored for ID: {payload_id}")

    return copy.deepcopy(_snapshot_store[payload_id][target_version])


def apply_rules(payload, rules):
    """
    Applies a list of transformation rules to the payload data.

    Args:
        payload (dict): A payload containing 'data'.
        rules (list): A list of rule dictionaries.

    Returns:
        dict: The transformed payload.
    """
    data = copy.deepcopy(payload["data"])

    for rule in rules:
        op = rule.get("op")
        if op in {"rename", "add", "remove", "set"}:
            data = apply_field_rule(data, rule)
        elif op == "map":
            data = apply_map_rule(data, rule)
        else:
            raise ValueError(f"Unsupported operation: {op}")

    return {"version": payload["version"], "data": data}


def apply_field_rule(data, rule):
    """
    Applies a single field-level rule to a data dictionary.

    Args:
        data (dict): Data to modify.
        rule (dict): Rule dict.

    Returns:
        dict: Modified data.
    """
    new_data = copy.deepcopy(data)
    op = rule["op"]

    if op == "rename":
        from_field = rule["from"]
        to_field = rule["to"]
        if from_field in new_data:
            new_data[to_field] = new_data.pop(from_field)

    elif op == "add":
        field = rule["field"]
        if field not in new_data:
            new_data[field] = rule["value"]

    elif op == "remove":
        field = rule["field"]
        new_data.pop(field, None)

    elif op == "set":
        field = rule["field"]
        new_data[field] = rule["value"]

    return new_data


def apply_map_rule(data, rule):
    """
    Applies a map rule to each element in a list at the specified path.

    Args:
        data (dict): Payload data.
        rule (dict): Map rule with path and nested rules.

    Returns:
        dict: Modified data.
    """
    path = rule["path"]
    subrules = rule["rules"]
    keys = path.split(".")

    list_parent, list_key = resolve_list_parent(data, keys)
    if list_parent is None or list_key not in list_parent:
        return data

    list_items = list_parent[list_key]
    if not isinstance(list_items, list):
        return data

    new_list = []
    for item in list_items:
        new_item = copy.deepcopy(item)
        for subrule in subrules:
            new_item = apply_field_rule(new_item, subrule)
        new_list.append(new_item)

    list_parent[list_key] = new_list
    return data


def resolve_list_parent(data, keys):
    """
    Traverses the path and returns the parent object and final key of the list.

    Args:
        data (dict): Data dictionary.
        keys (list): Path split into keys.

    Returns:
        tuple: (parent_object, list_key) or (None, None) if path is invalid.
    """
    obj = data
    for key in keys[:-1]:
        if not isinstance(obj, dict) or key not in obj:
            return None, None
        obj = obj[key]

    return obj, keys[-1]

In [ ]:
# tests

import unittest
import copy
from main import migrate_payload, rollback_payload, _snapshot_store


class TestSchemaEvolution(unittest.TestCase):
    def setUp(self):
        _snapshot_store.clear()

    def test_upgrade_v1_to_v3(self):
        payload = {"version": "v1",
                   "data": {"user_id": 1, "items": [{"price_cents": 100}]}}
        rules = {
            "v1_to_v2": [
                {"op": "map", "path": "items", "rules": [
                    {"op": "rename", "from": "price_cents", "to": "price"},
                    {"op": "add", "field": "currency", "value": "USD"}]}],
            "v2_to_v3": [
                {"op": "rename", "from": "user_id", "to": "uid"}],
        }
        res = migrate_payload(payload, rules, "v3", payload_id="p1")
        self.assertEqual(res["version"], "v3")
        self.assertEqual(res["data"]["uid"], 1)
        self.assertEqual(res["data"]["items"][0],
                         {"price": 100, "currency": "USD"})

    def test_downgrade_v3_to_v1(self):
        payload = {"version": "v3", "data": {"uid": 1}}
        rules = {
            "v1_to_v2": [],
            "v2_to_v1": [],      # reverse step
            "v2_to_v3": [],
            "v3_to_v2": [],      # reverse step
        }
        res = migrate_payload(payload, rules, "v1", payload_id="p2")
        self.assertEqual(res["version"], "v1")

    def test_missing_rule_error(self):
        with self.assertRaises(ValueError):
            migrate_payload({"version": "v1", "data": {}},
                            {}, "v2", payload_id="p3")

    def test_snapshot_and_rollback(self):
        payload = {"version": "v1", "data": {"x": 1}}
        rules = {"v1_to_v2": [{"op": "set", "field": "x", "value": 2}]}
        migrate_payload(payload, rules, "v2", payload_id="p4")
        snap = rollback_payload("p4", "v1")
        self.assertEqual(snap, payload)

    def test_rollback_missing_version(self):
        with self.assertRaises(KeyError):
            rollback_payload("no_id", "v1")

    def test_migrate_to_same_version(self):
        payload = {"version": "v2", "data": {"a": 1}}
        migrate_payload(payload, {}, "v2", payload_id="p5")
        self.assertIn("v2", _snapshot_store["p5"])

    def test_rollback_to_current_version(self):
        payload = {"version": "v1", "data": {"a": 1}}
        migrate_payload(payload, {}, "v1", payload_id="p6")
        snap1 = rollback_payload("p6", "v1")
        snap2 = rollback_payload("p6", "v1")
        self.assertIsNot(snap1, snap2)

    def test_map_path_missing(self):
        payload = {"version": "v1", "data": {}}
        rules = {"v1_to_v2": [{"op": "map", "path": "items", "rules": [
            {"op": "add", "field": "foo", "value": "bar"}]}]}
        res = migrate_payload(payload, rules, "v2", payload_id="p7")
        self.assertEqual(res["data"], {})

    def test_map_empty_list(self):
        payload = {"version": "v1", "data": {"items": []}}
        rules = {"v1_to_v2": [{"op": "map", "path": "items", "rules": [
            {"op": "add", "field": "foo", "value": "bar"}]}]}
        res = migrate_payload(payload, rules, "v2", payload_id="p8")
        self.assertEqual(res["data"]["items"], [])

    def test_field_operations_chain(self):
        payload = {"version": "v1", "data": {}}
        rules = {"v1_to_v2": [
            {"op": "add", "field": "a", "value": 1},
            {"op": "set", "field": "a", "value": 2},
            {"op": "rename", "from": "a", "to": "b"},
            {"op": "remove", "field": "b"}]}
        res = migrate_payload(payload, rules, "v2", payload_id="p9")
        self.assertEqual(res["data"], {})

    def test_unknown_op(self):
        payload = {"version": "v1", "data": {}}
        rules = {"v1_to_v2": [{"op": "noop"}]}
        with self.assertRaises(ValueError):
            migrate_payload(payload, rules, "v2", payload_id="p10")

    def test_input_not_mutated(self):
        payload = {"version": "v1", "data": {"k": 1}}
        original = copy.deepcopy(payload)
        rules = {"v1_to_v2": [{"op": "set", "field": "k", "value": 2}]}
        migrate_payload(payload, rules, "v2", payload_id="p11")
        self.assertEqual(payload, original)


if __name__ == "__main__":
    unittest.main()



# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/edee1dc4-e175-47e3-a948-c35f0fd5a5f3?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67538%2Fview

#### Model code:

```python
from copy import deepcopy

# A dictionary to store the versioned payloads
versioned_snapshots = {}

def parse_version(version_str):
    """Extract the version number from the version string."""
    return int(version_str[1:])

def apply_rule(data, rule):
    """Apply a single rule to the data."""
    op = rule["op"]
    
    if op == "rename":
        old_name = rule["from"]
        new_name = rule["to"]
        if old_name in data:
            data[new_name] = data.pop(old_name)
    elif op == "add":
        field = rule["field"]
        value = rule["value"]
        if field not in data:
            data[field] = value
    elif op == "remove":
        field = rule["field"]
        data.pop(field, None)
    elif op == "set":
        field = rule["field"]
        value = rule["value"]
        data[field] = value
    elif op == "map":
        path = rule["path"]
        rules = rule["rules"]
        
        # Resolve the path to the list
        parts = path.split('.')
        list_data = data
        for part in parts:
            if part in list_data:
                list_data = list_data[part]
            else:
                # The path does not exist or the list is not present, so it's a no-op
                return
        
        # Apply rules to each element in the list
        if isinstance(list_data, list):
            for item in list_data:
                if isinstance(item, dict):
                    for sub_rule in rules:
                        apply_rule(item, sub_rule)

def apply_migration_steps(data, migration_rules, current_version, target_version):
    """Apply migration steps from the current to the target version."""
    current_v = parse_version(current_version)
    target_v = parse_version(target_version)
    
    while current_v < target_v:
        step_key = f"v{current_v}_to_v{current_v+1}"
        if step_key not in migration_rules:
            raise ValueError(f"No migration path from v{current_v} to v{current_v+1}")
        
        # Apply all rules for the current step
        for rule in migration_rules[step_key]:
            apply_rule(data, rule)
        
        current_v += 1
        # Snapshot the current state
        versioned_snapshots[f"{data.get('id')}_v{current_v}"] = {
            "version": f"v{current_v}",
            "data": deepcopy(data)
        }

def migrate_payload(payload, migration_rules, target_version, payload_id):
    """Migrate the payload to the
    # <Issue>: The model failed to generate a complete solution. Which is not usable.
```